# Lab 06: AIP with LangChain - Multi-Tenant Workflow Chains

## Business Context

You are building a **Multi-Tenant Marketing Platform** using LangChain that serves multiple enterprise clients. Your platform generates AI-powered marketing content through multi-step chains:

- **Tenant A**: B2B tech company needing campaign content workflows
- **Tenant B**: B2C retail company needing product description workflows

**Challenge**: Implement LangChain chains with Application Inference Profiles for per-tenant isolation and cost tracking.

## Learning Objectives
- Integrate Application Inference Profiles with LangChain
- Create tenant-specific LLM instances using Bedrock
- Build simple multi-step chains with AIP integration
- Track token usage and costs per tenant
- Verify per-tenant CloudWatch metrics isolation

## What's Different from Previous Labs?
- **Lab 03 (Boto3)**: Direct API calls, manual chain logic
- **Lab 04 (Strands)**: Agent framework with automatic tool orchestration
- **Lab 05 (LiteLLM)**: Gateway abstraction for routing
- **Lab 06 (LangChain)**: Declarative chains with automatic prompt composition

## Focus: Application Inference Profiles
This lab keeps complexity low and focuses on **AIP integration**, not advanced LangChain features.

In [ ]:
# Install required packages
!pip install --force-reinstall -q -r requirements.txt --quiet

## Section 1: Setup and Create Application Inference Profiles

First, let's set up our environment with boto3 (for AIP management) and LangChain (for chain workflows), then check for existing AIPs or create new ones.

In [ ]:
import boto3
import json
import time
from datetime import datetime
from typing import Dict, Any
from langchain_aws import ChatBedrock

# Import lab helpers
from lab_helpers.config import Region, ModelId
from lab_helpers.aip_manager import AIPManager

# Initialize AWS clients
bedrock_client = boto3.client('bedrock', region_name=Region)
sts = boto3.client('sts', region_name=Region)

print(f"✅ Initialized boto3 clients for region: {Region}")
print(f"✅ Initialized LangChain for chain workflows")
print(f"📋 Using Claude Sonnet 4 model: {ModelId}")

### 🏗️ Setup: AIP Manager and Tenant Configurations

Initialize the Application Inference Profile manager and define configurations for two tenants:
- **Tenant A (B2B Tech)**: SaaS marketing content generation
- **Tenant B (B2C Retail)**: Product description and marketing copy

Each tenant gets:
- Unique AIP for isolated model access
- Tags for cost tracking and billing
- Separate CloudWatch metrics dimensions

In [ ]:
# Initialize AIP Manager
aip_manager = AIPManager(bedrock_client)

# Define tenant configurations (same as previous labs)
TENANT_CONFIGS = {
    "tenant_a": {
        "name": "marketing-ai-tenant-a",
        "description": "Marketing AI AIP for Tenant A",
        "tags": {
            "TenantId": "tenant-a",
            "BusinessType": "B2B-Tech",
            "Environment": "production",
            "CostCenter": "marketing-ai-platform"
        }
    },
    "tenant_b": {
        "name": "marketing-ai-tenant-b", 
        "description": "Marketing AI AIP for Tenant B",
        "tags": {
            "TenantId": "tenant-b",
            "BusinessType": "B2C-Retail",
            "Environment": "production",
            "CostCenter": "marketing-ai-platform"
        }
    }
}

print("📋 Tenant configurations defined:")
for tenant_id, config in TENANT_CONFIGS.items():
    print(f"  - {tenant_id}: {config['description']}")

### 🔍 Check for Existing AIPs (Reuse from Previous Labs)

Before creating new Application Inference Profiles, let's check if they already exist from Lab 03-05.

**Strategy:**
1. Check if AIPs exist with the same names
2. If exists: **Reuse them** (saves time and avoids duplicates)
3. If not exists: **Create new ones** using the same logic as Lab 03

This approach ensures we can run Lab 06 independently or after previous labs.

In [ ]:
# Check for existing AIPs and reuse or create
tenant_aips = {}

for tenant_id, config in TENANT_CONFIGS.items():
    print(f"\n🔍 Checking AIP for {tenant_id}...")
    
    try:
        # Check if AIP already exists (possibly from Lab 03-05)
        existing_arn = aip_manager.check_aip_exists(config["name"])
        
        if existing_arn:
            print(f"✅ Found existing AIP (reusing from previous lab)")
            print(f"   ARN: {existing_arn}")
            tenant_aips[tenant_id] = existing_arn
        else:
            print(f"📝 AIP not found - creating new one...")
            
            # Prepare tags
            tag_list = []
            if config["tags"]:
                tag_list = [{"key": k, "value": v} for k, v in config["tags"].items()]
            
            # Get account ID
            account_id = sts.get_caller_identity()['Account']
            
            # Construct proper ARN
            MODEL_ARN = f"arn:aws:bedrock:{Region}:{account_id}:inference-profile/{ModelId}"
            
            # Create Application Inference Profile
            response = bedrock_client.create_inference_profile(
                inferenceProfileName=config["name"],
                description=config["description"],
                modelSource={"copyFrom": MODEL_ARN},
                tags=tag_list
            )
            
            aip_arn = response['inferenceProfileArn']
            tenant_aips[tenant_id] = aip_arn
            
            print(f"✅ Created new AIP for {tenant_id}")
            print(f"   Status: {response['status']}")
            print(f"   ARN: {aip_arn}")
            
    except Exception as e:
        print(f"❌ Error with AIP for {tenant_id}: {str(e)}")

print(f"\n📊 Summary: {len(tenant_aips)} Application Inference Profiles ready for LangChain")

## Section 2: Create Tenant-Specific LangChain LLM Instances

Now let's create LangChain ChatBedrock instances for each tenant, configured with their specific Application Inference Profile.

In [ ]:
# Create tenant-specific LangChain LLM instances
tenant_llms = {}

for tenant_id, aip_arn in tenant_aips.items():
    print(f"\n🤖 Creating LangChain LLM for {tenant_id}...")
    
    try:
        # Create ChatBedrock instance with tenant's AIP
        # This is the key pattern: use aip_arn instead of system model ID
        llm = ChatBedrock(
            model_id=aip_arn,  # ← Use AIP ARN for tenant isolation
            region_name=Region,
            model_kwargs={
                "temperature": 0.7,
                "max_tokens": 1000
            }
        )
        
        tenant_llms[tenant_id] = llm
        
        print(f"✅ Created LangChain LLM for {tenant_id}")
        print(f"   Model: ChatBedrock with AIP")
        print(f"   AIP ARN: {aip_arn}")
        
    except Exception as e:
        print(f"❌ Error creating LLM for {tenant_id}: {str(e)}")

print(f"\n📊 Summary: {len(tenant_llms)} tenant-specific LangChain LLMs ready")

### 📊 CloudWatch Monitoring Function

Same monitoring function as previous labs - works with all approaches.

In [ ]:
def monitor_aip_usage(tenant_aips, region):
    """
    Monitor CloudWatch metrics for Application Inference Profiles.
    Works with all approaches (Boto3, Strands, LiteLLM, LangChain, LangGraph).
    """
    from lab_helpers.cloudwatch import fetch_metrices, plot_graph

    print("📊 Fetching CloudWatch metrics for Application Inference Profiles...")
    print(f"Region: {region}")
    print(f"Time Range: Last 60 minutes")
    print("="*80)

    tenant_metrics = {}

    for tenant_id, aip_arn in tenant_aips.items():
        print(f"\n🏢 TENANT: {tenant_id.upper()}")
        print("="*50)
        print(f"AIP ARN: {aip_arn}")
        
        # Extract AIP ID from full ARN for CloudWatch ModelId dimension
        aip_id = aip_arn.split('/')[-1]
        print(f"AIP ID: {aip_id}")
        
        try:
            print(f"\n📊 METRICS FOR {tenant_id.upper()}:")
            response, input_token_response, output_token_response = fetch_metrices(
                Region=region,
                Period=60,
                Timedelta=60,
                Id=aip_id
            )
            
            tenant_metrics[tenant_id] = {
                'invocations': response,
                'input_tokens': input_token_response,
                'output_tokens': output_token_response
            }
            
            print(f"\n📈 USAGE PLOTS FOR {tenant_id.upper()}:")
            plot_graph(response, input_token_response, output_token_response)
            
            print("="*50)
                
        except Exception as e:
            print(f"⚠️ CloudWatch error for {tenant_id}: {str(e)}")
            print(f"💡 Metrics may take a few minutes to appear after chain invocations")
            print("="*50)
    
    return tenant_metrics

print("✅ CloudWatch monitoring function defined")

### 🔵 BEFORE Check: Baseline CloudWatch Metrics

Check CloudWatch metrics before running any chains to establish a baseline.

In [ ]:
# Check CloudWatch metrics BEFORE any chain invocations
print("🔵 BEFORE CHECK: Querying CloudWatch for baseline metrics...")
print("="*80)
print("Expected: Empty if this is your first LangChain run")
print("="*80 + "\n")

baseline_metrics = monitor_aip_usage(tenant_aips, Region)

print("\n" + "="*80)
print("✅ Baseline check complete!")
print("💡 Note: Empty metrics are expected if this is your first run")
print("="*80)

## Section 3: Define Simple Content Generation Chains

Create simple, focused chains for each tenant. We keep this intentionally simple to focus on **AIP integration**, not complex LangChain features.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define simple chains for each tenant
# Chain 1: Campaign Headline Generator (Tenant A - B2B)
headline_prompt = PromptTemplate(
    input_variables=["product", "audience"],
    template="""Create a compelling marketing headline for a {product} targeting {audience}.
    The headline should be attention-grabbing and convey clear value.
    Respond with just the headline, nothing else."""
)

# Chain 2: Product Description Generator (Tenant B - B2C)
description_prompt = PromptTemplate(
    input_variables=["product", "features"],
    template="""Write a compelling product description for {product}.
    Key features: {features}
    Keep it under 100 words, engaging and customer-focused.
    Respond with just the description, nothing else."""
)

print("✅ Content generation prompts defined")

### 🎯 Create Tenant-Specific Chains

Build simple chains that route through each tenant's AIP-backed LLM.

In [ ]:
# Create chains for each tenant using their AIP-backed LLM
tenant_chains = {}

print("\n🔗 Creating tenant-specific chains...")
print("="*80)

# Tenant A chain: Headline generation
if "tenant_a" in tenant_llms:
    tenant_a_chain = headline_prompt | tenant_llms["tenant_a"] | StrOutputParser()
    tenant_chains["tenant_a"] = {
        "chain": tenant_a_chain,
        "type": "headline_generator",
        "input_vars": {"product": "DevOps automation platform", "audience": "CTOs at enterprise companies"}
    }
    print("\n✅ Tenant A Chain: Headline Generator")
    print(f"   LLM: ChatBedrock with AIP")
    print(f"   Purpose: Generate marketing headlines for B2B tech products")

# Tenant B chain: Product description generation
if "tenant_b" in tenant_llms:
    tenant_b_chain = description_prompt | tenant_llms["tenant_b"] | StrOutputParser()
    tenant_chains["tenant_b"] = {
        "chain": tenant_b_chain,
        "type": "description_generator",
        "input_vars": {"product": "Sustainable Summer Fashion Collection", "features": "eco-friendly materials, hand-crafted, limited edition"}
    }
    print("\n✅ Tenant B Chain: Product Description Generator")
    print(f"   LLM: ChatBedrock with AIP")
    print(f"   Purpose: Generate product descriptions for B2C retail products")

print("\n" + "="*80)
print(f"📊 Created {len(tenant_chains)} tenant-specific chains")
print("="*80)

### 🚀 Execute Chains for Each Tenant

Run the chains and capture outputs with per-tenant tracking.

In [ ]:
# Execute chains for each tenant
chain_results = {}

print("\n🚀 Executing tenant-specific chains...")
print("="*80)

for tenant_id, chain_config in tenant_chains.items():
    print(f"\n🏢 Processing {tenant_id.upper()}")
    print("-"*50)
    
    try:
        # Record start time for latency calculation
        start_time = datetime.now()
        
        # Execute the chain with tenant-specific inputs
        output = chain_config["chain"].invoke(chain_config["input_vars"])
        
        # Calculate latency
        end_time = datetime.now()
        latency_ms = (end_time - start_time).total_seconds() * 1000
        
        # Store result
        chain_results[tenant_id] = {
            "type": chain_config["type"],
            "output": output,
            "latency_ms": latency_ms,
            "timestamp": datetime.now().isoformat(),
            "success": True
        }
        
        print(f"✅ Chain executed for {tenant_id}")
        print(f"   Output: {output[:80]}...")
        print(f"   Latency: {latency_ms:.2f}ms")
        
    except Exception as e:
        print(f"❌ Error executing chain for {tenant_id}: {str(e)}")
        chain_results[tenant_id] = {
            "error": str(e),
            "timestamp": datetime.now().isoformat(),
            "success": False
        }

print(f"\n" + "="*80)
print(f"📊 Executed {len([r for r in chain_results.values() if r['success']])} successful chains")
print("="*80)

## Section 4: Review Results and CloudWatch Metrics

Display the generated outputs and verify per-tenant CloudWatch metrics.

### 📄 Display Chain Outputs

In [ ]:
# Display chain outputs
print("📄 CHAIN OUTPUTS")
print("="*80)

for tenant_id, result in chain_results.items():
    if result['success']:
        print(f"\n🏢 {tenant_id.upper()} - {result['type']}")
        print("-"*50)
        print(f"Output:\n{result['output']}")
        print(f"\nLatency: {result['latency_ms']:.2f}ms")
        print("="*80)
    else:
        print(f"\n❌ {tenant_id.upper()}: {result['error']}")
        print("="*80)

### 🟢 AFTER Check: CloudWatch Metrics Show Multi-Tenant Isolation

Now let's check CloudWatch metrics after chain execution to verify per-tenant isolation.

In [ ]:
# Wait for CloudWatch metrics to propagate
print("⏳ Waiting 60 seconds for CloudWatch metrics to propagate...")
time.sleep(60)
print("✅ Wait complete - proceeding with metrics monitoring\n")

# Check CloudWatch metrics AFTER chain execution
print("🟢 AFTER CHECK: Querying CloudWatch for post-chain metrics...")
print("="*80)
print("Expected: Per-tenant metrics showing isolated usage!")
print("="*80 + "\n")

after_metrics = monitor_aip_usage(tenant_aips, Region)

print("\n" + "="*80)
print("✅ Metrics check complete!")
print("💡 If you see metrics data, multi-tenant isolation is working!")
print("   Each tenant's chain execution tracked separately via their AIP.")
print("="*80)

### 📊 Before/After Comparison

Summary of the multi-tenant isolation proof through LangChain + AIP.

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════════════════╗
║              LANGCHAIN + AIP: Multi-Tenant Isolation Proof                   ║
╚════════════════════════════════════════════════════════════════════════════╝

🔵 BEFORE (Baseline Check):
   Status: AIPs created and configured for LangChain
   Chains: Ready but unused
   CloudWatch: No metrics (AIPs unused)
   Tenant Isolation: Ready but unverified

🟢 AFTER (Post-Chain Execution):
   Chains: ✅ Executed via tenant-specific AIP-backed LLMs
   Outputs: ✅ Generated per-tenant content
   CloudWatch Metrics: ✅ Separate dimensions per tenant
   Cost Tracking: ✅ Per-tenant billing enabled

📊 WHAT WE PROVED:
   1. ✅ LangChain ChatBedrock integrates with AIPs
   2. ✅ Each tenant's chain execution tracked separately in CloudWatch
   3. ✅ Simple prompt → LLM → output pattern scales to N tenants
   4. ✅ AIP provides transparent per-tenant isolation in LangChain
   5. ✅ Accurate cost allocation with LangChain chains

💡 THE PATTERN:
   ChatBedrock(model_id=aip_arn)  →  Prompt | LLM | Parser  →  CloudWatch
   (Tenant-specific)                (Chain execution)            (Isolated metrics)

✨ KEY INSIGHT:
   LangChain chains + AIPs = Production-ready multi-tenant content generation
""")

## Lab Summary

🎉 **Congratulations!** You've successfully integrated Application Inference Profiles with LangChain chains for a multi-tenant content generation platform.

### What You Accomplished:

1. **✅ Checked and reused existing AIPs** from previous labs
2. **✅ Created tenant-specific ChatBedrock LLM instances** using AIP ARNs
3. **✅ Built simple LangChain prompt chains** with AIP integration
4. **✅ Checked CloudWatch metrics BEFORE** chain execution (baseline/empty state)
5. **✅ Executed tenant-specific chains** with isolated routing
6. **✅ Checked CloudWatch metrics AFTER** chain execution (per-tenant usage visible!)
7. **✅ Proved multi-tenant isolation** works with LangChain

### Key Takeaways:

- **AIP Integration**: ChatBedrock + aip_arn enables automatic per-tenant tracking
- **Chain Simplicity**: Prompt | LLM | Parser pattern is clean and composable
- **Tenant Isolation**: CloudWatch metrics remain separate despite LangChain abstraction
- **Cost Tracking**: Built-in token usage enables accurate billing
- **Multi-Step Workflows**: Can compose multiple chains for complex workflows

### The Pattern:

```python
# For each tenant:
llm = ChatBedrock(model_id=aip_arn)  # AIP provides isolation
chain = prompt | llm | parser        # Simple declarative chain
output = chain.invoke(inputs)        # Automatic CloudWatch tracking
```

### Framework Comparison:

| Lab | Framework | Code Simplicity | Flexibility | Best For |
|-----|-----------|-----------------|-------------|----------|
| Lab 03 | Boto3 | Low | High | Fine-grained control |
| Lab 04 | Strands | Medium | Medium | Agents with tools |
| Lab 05 | LiteLLM | High | Low | Gateway patterns |
| **Lab 06** | **LangChain** | **High** | **Medium** | **Simple chains & composition** |
| Lab 07 | LangGraph | High | High | Complex workflows |

### Next Steps:

Continue your AIP journey with graph-based workflows:
- **Lab 07**: LangGraph - Graph workflows for complex multi-step processes

**Ready for complex workflows?** → [Continue to Lab 07: LangGraph](lab-07-aip-langgraph-optional.ipynb)